In [1]:
from train_test_fns import *
import re

In [33]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [157]:
pl2 = players = pd.read_csv('baseballdatabank-master/core/People.csv')

In [19]:
train,test = load_data()

In [20]:
train = engineer_features(train)

In [6]:
train['name'][0]

'Jeff Abbott'

In [60]:
batting = pd.read_csv('baseballdatabank-master/core/Batting.csv')
bat_2000 = batting[batting['yearID'] >= 2000]
bat_2000_over_100 = bat_2000[bat_2000['AB']>100]
#bat_2000_over_100 = bat_2000_over_100.set_index('playerID')

In [131]:
players = pd.read_csv('baseballdatabank-master/core/People.csv')
#players = players.set_index('playerID')
players = players.drop(['deathYear','deathMonth','deathDay','deathCountry','deathState','deathCity'],axis=1)
players['fullname'] = players['nameFirst'] + ' ' + players['nameLast']
players = players.dropna()

players['fullname'] = players['fullname'].apply(lambda x: ''.join(re.sub(r'[^\w\s]','',x).split(' ')).lower())

In [130]:
''.join('jeff abbot'.split(' '))

'jeffabbot'

In [132]:
bat_join = bat_2000_over_100.merge(players,how='left',on='playerID')

In [133]:
bat_join.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,fullname
0,abbotje01,2000,1,CHA,AL,80,215,31,59,15,1,3,29.0,2.0,1.0,21,38.0,1.0,2.0,2.0,1.0,2.0,1972.0,8.0,17.0,USA,GA,Atlanta,Jeff,Abbott,Jeffrey William,190.0,74.0,R,L,1997-06-10,2001-09-29,abboj002,abbotje01,jeffabbott
1,abbotku01,2000,1,NYN,NL,79,157,22,34,7,1,6,12.0,1.0,1.0,14,51.0,2.0,1.0,0.0,1.0,2.0,1969.0,6.0,2.0,USA,OH,Zanesville,Kurt,Abbott,Kurt Thomas,180.0,71.0,R,R,1993-09-07,2001-04-13,abbok002,abbotku01,kurtabbott
2,abreubo01,2000,1,PHI,NL,154,576,103,182,42,10,25,79.0,28.0,8.0,100,116.0,9.0,1.0,0.0,3.0,12.0,1974.0,3.0,11.0,Venezuela,Aragua,Maracay,Bobby,Abreu,Bob Kelly,220.0,72.0,L,R,1996-09-01,2014-09-28,abreb001,abreubo01,bobbyabreu
3,agbaybe01,2000,1,NYN,NL,119,350,59,101,19,1,15,60.0,5.0,5.0,54,68.0,2.0,7.0,0.0,3.0,6.0,1971.0,12.0,28.0,USA,HI,Honolulu,Benny,Agbayani,Benny Peter,225.0,72.0,R,R,1998-06-17,2002-09-29,agbab001,agbaybe01,bennyagbayani
4,alexama02,2000,1,BOS,AL,101,194,30,41,4,3,4,19.0,2.0,0.0,13,41.0,0.0,0.0,2.0,0.0,0.0,1971.0,3.0,20.0,D.R.,San Pedro de Macoris,San Pedro de Macoris,Manny,Alexander,Manuel de Jesus,150.0,70.0,R,R,1992-09-18,2006-09-28,alexm001,alexama02,mannyalexander


In [134]:
keep_cols = ['yearID',
 'G',
 'AB',
 'R',
 'H',
 '2B',
 '3B',
 'HR',
 'RBI',
 'SB',
 'CS',
 'BB',
 'SO',
 'IBB',
 'HBP',
 'SH',
 'SF',
 'GIDP',
 'debut',
 'fullname']

In [136]:
bat_join = bat_join[keep_cols]

bat_join.columns = [x.lower() for x in bat_join.columns]

In [147]:
bat_join = bat_join.groupby(['fullname','yearid','debut'],axis=0)['g','ab','r','h','2b','3b','hr','rbi','sb','cs','bb','so','ibb','hbp','sh','sf','gidp'].sum().reset_index()
bat_join['str_g'] = bat_join['g'].apply(str)
bat_join['str_year'] = bat_join['yearid'].apply(str)
bat_join['name_g_y'] = bat_join['fullname'] + ' ' + bat_join['str_g'] + ' ' + bat_join['str_year']

bat_join.head()

,fullname,yearid,debut,g,ab,r,h,2b,3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp,str_g,str_year,name_g_y
0,aaronaltherr,2015,2014-06-16,39,137,25,33,11,4,5,22.0,6.0,2.0,16,41.0,0.0,5.0,1.0,2.0,3.0,39,2015,aaronaltherr 39 2015
1,aaronaltherr,2016,2014-06-16,57,198,23,39,6,0,4,22.0,7.0,2.0,23,69.0,2.0,6.0,0.0,0.0,4.0,57,2016,aaronaltherr 57 2016
2,aaronaltherr,2017,2014-06-16,107,372,58,101,24,5,19,65.0,5.0,4.0,32,104.0,2.0,7.0,0.0,1.0,12.0,107,2017,aaronaltherr 107 2017
3,aaronboone,2000,1997-06-20,84,291,44,83,18,0,12,43.0,6.0,1.0,24,52.0,1.0,10.0,2.0,4.0,5.0,84,2000,aaronboone 84 2000
4,aaronboone,2001,1997-06-20,103,381,54,112,26,2,14,62.0,6.0,3.0,29,71.0,1.0,8.0,3.0,6.0,9.0,103,2001,aaronboone 103 2001


In [163]:
bat_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7193 entries, 0 to 7192
Data columns (total 23 columns):
fullname    7193 non-null object
yearid      7193 non-null int64
debut       7193 non-null object
g           7193 non-null int64
ab          7193 non-null int64
r           7193 non-null int64
h           7193 non-null int64
2b          7193 non-null int64
3b          7193 non-null int64
hr          7193 non-null int64
rbi         7193 non-null float64
sb          7193 non-null float64
cs          7193 non-null float64
bb          7193 non-null int64
so          7193 non-null float64
ibb         7193 non-null float64
hbp         7193 non-null float64
sh          7193 non-null float64
sf          7193 non-null float64
gidp        7193 non-null float64
str_g       7193 non-null object
str_year    7193 non-null object
name_g_y    7193 non-null object
dtypes: float64(9), int64(9), object(5)
memory usage: 1.3+ MB


In [164]:
bat_join[bat_join['name_g_y'].str.contains('alecia')]

,fullname,yearid,debut,g,ab,r,h,2b,3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp,str_g,str_year,name_g_y


In [149]:
train['str_g'] = train['g'].apply(str)
train['str_year'] = train['year'].apply(str)
train['name_g_y'] = train['name'].apply(lambda x: ''.join(x.split(' ')).lower()) + ' ' + train['str_g'] + ' ' + train['str_year']
#train = train.set_index('name_g_y')
train.head()

,name,age,g,pa,rbat,rbaser,rdp,rfield,rpos,raa,waa,rrep,rar,war,waawl%,162wl%,owar,dwar,orar,salary,year,adj_salary,log10_adj,acquired_dr,acquired_fa,bat_hand_lhb,bat_hand_swh,tm_ARI,tm_ATL,tm_BAL,tm_BOS,tm_CHC,tm_CHW,tm_CIN,tm_CLE,tm_COL,tm_DET,tm_HOU,tm_KCR,tm_LAA,tm_LAD,tm_MIA,tm_MIL,tm_MIN,tm_NYM,tm_NYY,tm_OAK,tm_PHI,tm_PIT,tm_SDP,tm_SEA,tm_SFG,tm_STL,tm_TBR,tm_TEX,tm_TOR,tm_WSN,str_g,str_year,name_g_y
0,Jeff Abbott,27,80,242,-7,1,1,-6,-1,-13,-1.1,9,-3,-0.3,0.486,0.493,0.3,-0.6,2,255000.0,2000,362982,5.559885,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,2000,jeffabbott 80 2000
1,Bobby Abreu,26,154,680,38,-2,1,15,-6,46,4.2,21,67,6.2,0.528,0.527,4.9,0.8,52,2933333.0,2000,4175485,6.620707,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,154,2000,bobbyabreu 154 2000
2,Benny Agbayani,28,119,415,13,0,0,0,-4,10,0.8,13,22,2.0,0.508,0.506,2.1,-0.5,23,220000.0,2000,313161,5.495768,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,119,2000,bennyagbayani 119 2000
3,Manny Alexander,29,101,209,-18,1,1,-2,2,-16,-1.5,8,-8,-0.7,0.485,0.491,-0.6,0.1,-6,612500.0,2000,871869,5.940451,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101,2000,mannyalexander 101 2000
4,Edgardo Alfonzo,26,150,650,46,1,-1,1,4,50,4.5,20,70,6.4,0.531,0.529,6.4,0.4,69,4225000.0,2000,6014123,6.779172,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,150,2000,edgardoalfonzo 150 2000


In [ ]:
test_1 = train['name_g_y'][0]
test_2 = train['name_g_y'][1]


In [ ]:
s = test_1.split(' ')[0]
t = test_2.split(' ')[0]

In [ ]:
(' ').join(s.split('\xa0'))

In [ ]:
t

In [150]:
merged = train.merge(bat_join,how='left',on='name_g_y')

In [152]:
merged[merged['fullname'].isnull()]

,name,age,g_x,pa,rbat,rbaser,rdp,rfield,rpos,raa,waa,rrep,rar,war,waawl%,162wl%,owar,dwar,orar,salary,year,adj_salary,log10_adj,acquired_dr,acquired_fa,bat_hand_lhb,bat_hand_swh,tm_ARI,tm_ATL,tm_BAL,tm_BOS,tm_CHC,tm_CHW,tm_CIN,tm_CLE,tm_COL,tm_DET,tm_HOU,tm_KCR,tm_LAA,tm_LAD,tm_MIA,tm_MIL,tm_MIN,tm_NYM,tm_NYY,tm_OAK,tm_PHI,tm_PIT,tm_SDP,tm_SEA,tm_SFG,tm_STL,tm_TBR,tm_TEX,tm_TOR,tm_WSN,str_g_x,str_year_x,name_g_y,fullname,yearid,debut,g_y,ab,r,h,2b,3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp,str_g_y,str_year_y
5,Luis Alicea,34,139,619,-4,1,-1,-11,3,-12,-1.0,23,11,1.1,0.492,0.493,2.1,-0.6,21,750000.0,2000,1067595,6.028407,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,139,2000,luisalicea 139 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Roberto Alomar,32,155,697,16,7,0,7,4,35,3.3,25,60,5.6,0.520,0.520,4.9,1.1,53,7196656.0,2000,10244159,7.010476,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,155,2000,robertoalomar 155 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Sandy Alomar,34,97,384,-9,0,0,-5,6,-9,-0.8,14,5,0.6,0.491,0.495,1.0,0.1,10,2700000.0,2000,3843344,6.584709,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,97,2000,sandyalomar 97 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Harold Baines,41,96,320,-6,-1,-1,0,-7,-15,-1.4,12,-3,-0.3,0.485,0.495,-0.3,-0.6,-3,2000000.0,2000,2846922,6.454376,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96,2000,haroldbaines 96 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Paul Bako,28,81,251,-17,-1,-1,0,4,-14,-1.5,8,-7,-0.7,0.483,0.495,-0.7,0.3,-6,275000.0,2000,391451,5.592677,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,81,2000,paulbako 81 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Danny Bautista,28,131,388,-2,-1,1,-1,-4,-6,-0.7,12,6,0.5,0.495,0.498,0.6,-0.5,7,500000.0,2000,711730,5.852315,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,131,2000,dannybautista 131 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Rich Becker,28,115,357,4,1,1,-9,-3,-6,-0.5,13,8,0.8,0.496,0.497,1.6,-1.0,17,700000.0,2000,996422,5.998443,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,115,2000,richbecker 115 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Carlos Beltran,23,98,413,-19,4,0,7,1,-7,-0.6,15,9,0.9,0.494,0.496,0.2,0.8,1,350000.0,2000,498211,5.697413,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,98,2000,carlosbeltran 98 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,Wil Cordero,28,127,536,3,-3,-1,-17,-5,-22,-2.2,18,-5,-0.5,0.483,0.493,1.2,-2.1,13,2166667.0,2000,3084166,6.489138,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,127,2000,wilcordero 127 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,Jose Cruz,26,162,681,-9,1,1,4,4,0,0.1,26,26,2.5,0.500,0.500,2.2,0.7,22,395000.0,2000,562267,5.749943,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,162,2000,josecruz 162 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
train[train['name'].str.contains('Alicea')]

,name,age,g,pa,rbat,rbaser,rdp,rfield,rpos,raa,waa,rrep,rar,war,waawl%,162wl%,owar,dwar,orar,salary,year,adj_salary,log10_adj,acquired_dr,acquired_fa,bat_hand_lhb,bat_hand_swh,tm_ARI,tm_ATL,tm_BAL,tm_BOS,tm_CHC,tm_CHW,tm_CIN,tm_CLE,tm_COL,tm_DET,tm_HOU,tm_KCR,tm_LAA,tm_LAD,tm_MIA,tm_MIL,tm_MIN,tm_NYM,tm_NYY,tm_OAK,tm_PHI,tm_PIT,tm_SDP,tm_SEA,tm_SFG,tm_STL,tm_TBR,tm_TEX,tm_TOR,tm_WSN,str_g,str_year,name_g_y
5,Luis Alicea,34,139,619,-4,1,-1,-11,3,-12,-1.0,23,11,1.1,0.492,0.493,2.1,-0.6,21,750000.0,2000,1067595,6.028407,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,139,2000,luisalicea 139 2000
339,Luis Alicea,35,113,418,-12,0,0,-1,0,-13,-1.2,16,3,0.3,0.489,0.492,0.3,0.0,3,800000.0,2001,1107449,6.044324,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,113,2001,luisalicea 113 2001
652,Luis Alicea,36,94,273,-13,0,0,2,0,-11,-1.1,10,-1,-0.1,0.488,0.493,-0.3,0.2,-3,800000.0,2002,1090148,6.037485,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,94,2002,luisalicea 94 2002


In [154]:
bat_join[bat_join['fullname'].str.contains('alicea')]

,fullname,yearid,debut,g,ab,r,h,2b,3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp,str_g,str_year,name_g_y


In [161]:
test1 = batting[batting['playerID'] == 'alicelu01']

In [162]:
test2 = pl2[pl2['nameLast'] == 'Alicea']

In [165]:
test1

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
65327,alicelu01,1988,1,SLN,NL,93,297,20,63,10,4,1,24.0,1.0,1.0,25,32.0,4.0,2.0,4.0,2.0,12.0
68553,alicelu01,1991,1,SLN,NL,56,68,5,13,3,0,0,0.0,0.0,1.0,8,19.0,0.0,0.0,0.0,0.0,0.0
69637,alicelu01,1992,1,SLN,NL,85,265,26,65,9,11,2,32.0,2.0,5.0,27,40.0,1.0,4.0,2.0,4.0,5.0
70700,alicelu01,1993,1,SLN,NL,115,362,50,101,19,3,3,46.0,11.0,1.0,47,54.0,2.0,4.0,1.0,7.0,9.0
71876,alicelu01,1994,1,SLN,NL,88,205,32,57,12,5,5,29.0,4.0,5.0,30,38.0,4.0,3.0,1.0,3.0,1.0
72916,alicelu01,1995,1,BOS,AL,132,419,64,113,20,3,6,44.0,13.0,10.0,63,61.0,0.0,7.0,13.0,9.0,10.0
74170,alicelu01,1996,1,SLN,NL,129,380,54,98,26,3,5,42.0,11.0,3.0,52,78.0,10.0,5.0,4.0,6.0,4.0
75422,alicelu01,1997,1,ANA,AL,128,388,59,98,16,7,5,37.0,22.0,8.0,69,65.0,3.0,8.0,4.0,2.0,4.0
76658,alicelu01,1998,1,TEX,AL,101,259,51,71,15,3,6,33.0,4.0,3.0,37,40.0,0.0,5.0,4.0,3.0,1.0
77980,alicelu01,1999,1,TEX,AL,68,164,33,33,10,0,3,17.0,2.0,1.0,28,32.0,0.0,0.0,3.0,1.0,4.0


In [166]:
test2

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
190,alicelu01,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01


In [170]:
test1.merge(test2,how='left',on='playerID')

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,alicelu01,1988,1,SLN,NL,93,297,20,63,10,4,1,24.0,1.0,1.0,25,32.0,4.0,2.0,4.0,2.0,12.0,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01
1,alicelu01,1991,1,SLN,NL,56,68,5,13,3,0,0,0.0,0.0,1.0,8,19.0,0.0,0.0,0.0,0.0,0.0,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01
2,alicelu01,1992,1,SLN,NL,85,265,26,65,9,11,2,32.0,2.0,5.0,27,40.0,1.0,4.0,2.0,4.0,5.0,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01
3,alicelu01,1993,1,SLN,NL,115,362,50,101,19,3,3,46.0,11.0,1.0,47,54.0,2.0,4.0,1.0,7.0,9.0,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01
4,alicelu01,1994,1,SLN,NL,88,205,32,57,12,5,5,29.0,4.0,5.0,30,38.0,4.0,3.0,1.0,3.0,1.0,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01
5,alicelu01,1995,1,BOS,AL,132,419,64,113,20,3,6,44.0,13.0,10.0,63,61.0,0.0,7.0,13.0,9.0,10.0,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01
6,alicelu01,1996,1,SLN,NL,129,380,54,98,26,3,5,42.0,11.0,3.0,52,78.0,10.0,5.0,4.0,6.0,4.0,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01
7,alicelu01,1997,1,ANA,AL,128,388,59,98,16,7,5,37.0,22.0,8.0,69,65.0,3.0,8.0,4.0,2.0,4.0,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01
8,alicelu01,1998,1,TEX,AL,101,259,51,71,15,3,6,33.0,4.0,3.0,37,40.0,0.0,5.0,4.0,3.0,1.0,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01
9,alicelu01,1999,1,TEX,AL,68,164,33,33,10,0,3,17.0,2.0,1.0,28,32.0,0.0,0.0,3.0,1.0,4.0,1965.0,7.0,29.0,P.R.,NaN,Santurce,NaN,NaN,NaN,NaN,NaN,NaN,Luis,Alicea,Luis Rene,165.0,69.0,B,R,1988-04-23,2002-09-25,alicl001,alicelu01


In [155]:
players[players['nameLast'].str.contains('Alicea')]

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,fullname


In [59]:
bat_join.groupby(['fullname','yearid','debut'],axis=0)['g','ab','r','h','2b','3b','hr','rbi','sb','cs','bb','so','ibb','hbp','sh','sf','gidp'].sum().reset_index()

,fullname,yearid,debut,g,ab,r,h,2b,3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
0,A. J. Burnett,2000,1999-08-17,13,25,3,7,1,1,1,3.0,0.0,0.0,3,10.0,0.0,0.0,2.0,0.0,0.0
1,A. J. Burnett,2001,1999-08-17,27,50,0,4,1,0,0,0.0,0.0,0.0,1,27.0,0.0,1.0,7.0,0.0,0.0
2,A. J. Burnett,2002,1999-08-17,31,57,1,6,2,0,1,3.0,0.0,0.0,5,28.0,0.0,0.0,7.0,0.0,1.0
3,A. J. Burnett,2003,1999-08-17,4,7,0,1,0,0,0,0.0,0.0,0.0,2,3.0,0.0,0.0,0.0,0.0,0.0
4,A. J. Burnett,2004,1999-08-17,20,29,4,4,0,0,0,1.0,0.0,0.0,0,11.0,0.0,0.0,8.0,0.0,1.0
5,A. J. Burnett,2005,1999-08-17,32,68,3,10,2,2,1,2.0,0.0,0.0,1,34.0,0.0,1.0,9.0,0.0,1.0
6,A. J. Burnett,2006,1999-08-17,21,3,0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
7,A. J. Burnett,2007,1999-08-17,25,2,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
8,A. J. Burnett,2008,1999-08-17,35,3,0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,1.0,0.0,0.0
9,A. J. Burnett,2009,1999-08-17,33,5,0,1,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
